In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Sep 10 18:49:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    34W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.4 MB/s 
     |████████████████████████████████| 81 kB 12.0 MB/s 
     |████████████████████████████████| 209 kB 70.6 MB/s 
     |████████████████████████████████| 78 kB 10.0 MB/s 
     |████████████████████████████████| 112 kB 76.6 MB/s 
     |████████████████████████████████| 147 kB 72.8 MB/s 
     |████████████████████████████████| 49 kB 8.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=d4a844514a30f436a85d610c4a5c163fa7ab0d543495de8c30bd38bd0872053b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:

import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

[I 2022-09-11 00:02:21,562] A new study created in memory with name: no-name-5e747c0f-691b-47dd-bd81-d14f2cb184cc
[I 2022-09-11 00:02:21,570] Trial 0 finished with value: 10.15610963081252 and parameters: {'x': -1.1868651729893624}. Best is trial 0 with value: 10.15610963081252.
[I 2022-09-11 00:02:21,576] Trial 1 finished with value: 7.058751917684265 and parameters: {'x': 4.656831179748586}. Best is trial 1 with value: 7.058751917684265.
[I 2022-09-11 00:02:21,581] Trial 2 finished with value: 0.01366490147376274 and parameters: {'x': 2.1168969694806616}. Best is trial 2 with value: 0.01366490147376274.
[I 2022-09-11 00:02:21,586] Trial 3 finished with value: 133.3396467575259 and parameters: {'x': -9.5472787598432}. Best is trial 2 with value: 0.01366490147376274.
[I 2022-09-11 00:02:21,591] Trial 4 finished with value: 25.983246746925413 and parameters: {'x': -3.0973764572498874}. Best is trial 2 with value: 0.01366490147376274.
[I 2022-09-11 00:02:21,596] Trial 5 finished with val

{'x': 1.9987105668553622}

In [ ]:
!conda

/bin/bash: conda: command not found


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

(#3) [Path('train'),Path('valid'),Path('labels.csv')]

In [ ]:
(path/'train').ls()

(#2) [Path('train/3'),Path('train/7')]

In [ ]:
(path/'train/3').ls()

(#6131) [Path('train/3/5547.png'),Path('train/3/55071.png'),Path('train/3/29972.png'),Path('train/3/28458.png'),Path('train/3/25809.png'),Path('train/3/22322.png'),Path('train/3/35272.png'),Path('train/3/6585.png'),Path('train/3/9871.png'),Path('train/3/37928.png')...]

In [ ]:
threes = [Image.open(x) for x in (path/'train/3').ls()]
sevens = [Image.open(x) for x in (path/'train/7').ls()]
threes_v = [Image.open(x) for x in (path/'valid/3').ls()]
sevens_v = [Image.open(x) for x in (path/'valid/7').ls()]

In [ ]:
sevens_v[3]

In [ ]:
t3 = torch.stack([tensor(x) for x in threes])
t7 = torch.stack([tensor(x) for x in sevens])
y3 = tensor([0] * len(threes))
y7 = tensor([1] * len(sevens))

In [ ]:
t3_v = torch.stack([tensor(x) for x in threes_v])
t7_v = torch.stack([tensor(x) for x in sevens_v])
y3_v = tensor([0] * len(threes_v))
y7_v = tensor([1] * len(sevens_v))

In [ ]:
y = torch.cat((y3, y7))
y.sum()

tensor(6265)

In [ ]:
images = torch.cat((t3, t7), 0).view(-1, 28 * 28).float() / 255
images.shape

torch.Size([12396, 784])

In [ ]:
weights = tensor([0.00001] * (28 * 28)).requires_grad_()
weights.shape

torch.Size([784])

In [ ]:
torch.randn((3,1))

tensor([[-1.9607],
        [ 0.1263],
        [ 1.4422]])

In [ ]:
bias = tensor(.1).requires_grad_()

In [ ]:
data_set = list(zip(images, y))

In [ ]:
images_v = torch.cat((t3_v, t7_v), 0).view(-1, 28 * 28).float() / 255
data_set_v = list(zip(images_v, torch.cat((y3_v, y7_v))))

In [ ]:
dl = DataLoader(data_set, batch_size=256, shuffle=True)
dl_v = DataLoader(data_set, batch_size=256, shuffle=True)

In [ ]:
torch.randn(20)

tensor([-0.5067, -0.6826,  1.2044,  2.0234, -0.3320,  0.5984,  0.0972,  0.1731,  1.0078, -0.7497,  0.8628, -1.1609,  0.9058,  0.5813,  1.3356, -0.1341,  0.0099,  0.7730, -0.3021,  0.4013])

In [ ]:
1.0 / (1 + np.exp(-10))

0.9999546021312976

In [ ]:
a = torch.Tensor(3)
a

tensor([2.2667e-36, 0.0000e+00, 3.3429e-34])

In [ ]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

def validate(weights, bias, dl):
  accuracy = tensor([])

  for batch in dl:
    [image_batch, y_batch] = batch
    predictions = (image_batch @ weights + bias).sigmoid()    
    accuracy = torch.cat((accuracy, torch.where(y_batch == 1, predictions >= 0.5, predictions < 0.5).float()), 0)
  return round(accuracy.mean().item(), 4)

def train_epoch(weights, bias, dl):
  epochs = 20
  accuracy = tensor([])
  for i in range(epochs):
    for batch in dl:
      [image_batch, y_batch] = batch
      predictions = (image_batch @ weights + bias).sigmoid()
      # print('predictions', (image_batch @ weights + bias)[0:10])
      # print(image_batch[0])
      # print(image_batch @ weights + bias)
      errors = torch.where(y_batch == 1, 1 - predictions, predictions)
      
      accuracy = torch.cat((accuracy, torch.where(y_batch == 1, predictions >= 0.5, predictions < 0.5).float()), 0)
      # accuracy = torch.where(y_batch == 1, predictions >= 0.5, predictions < 0.5).float()

      # print('accuracy current', accuracy.mean())
      loss = errors.mean()
      # print('loss', loss)
      
      loss.backward()
      step = 1

      # print('bias', bias.data, 'grad', bias.grad, 'weights', weights.data.mean(), 'grad', weights.grad.mean())
      # print('weights', weights[0:10])
      weights.data -=  weights.grad.data * step
      # print('bias before', bias.data, bias.grad.data)
      bias.data -= bias.grad.data * step
      
      
      weights.grad = None
      bias.grad = None

    print(validate(weights, bias, dl_v), end = ' ')

weights = init_params((28 * 28))
bias = init_params(1)
train_epoch(weights, bias, dl)


0.8215 0.9229 0.9422 0.9508 0.957 0.961 0.964 0.9657 0.9672 0.9688 0.9693 0.9706 0.9719 0.9729 0.9742 0.9744 0.9748 0.9752 0.9756 0.9764 